In [57]:
# Interesting links: 

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
# https://medium.com/swlh/sentiment-classification-for-restaurant-reviews-using-tf-idf-42f707bfe44d
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
# https://medium.com/analytics-vidhya/complete-guide-to-machine-learning-evaluation-metrics-615c2864d916

In [58]:
pip install nltk pycld2 wordsegment autocorrect

In [59]:
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
twitter_samples.fileids()
positive_tweets = [t for t in twitter_samples.strings("positive_tweets.json")]
negative_tweets = [t for t in twitter_samples.strings("negative_tweets.json")] 

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
print(vectorizer.get_feature_names())
print(X.shape)
print(X)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
(4, 9)
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [61]:
print(len(positive_tweets))
print(len(negative_tweets))

5000
5000


In [62]:
train_pos_tweets = positive_tweets[:3500]
test_pos_tweets = positive_tweets[3500:]
train_neg_tweets = negative_tweets[:3500]
test_neg_tweets = negative_tweets[3500:]

In [63]:
print(train_pos_tweets[0])

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [64]:
print(train_neg_tweets[7])

@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln


In [65]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [66]:
print(len(train_pos_tweets + train_neg_tweets))
print(len(test_pos_tweets + test_neg_tweets))

7000
3000


In [67]:
# import pycld2 as cld2

# def extract_engTweets(tweets):
#     en_tweets = []
#     for e in tweets:
#       details = cld2.detect(e)
#       # Extracts only the string with lang name i.e: 'en'
#       if details[2][0][1] == 'en': 
#         en_tweets.append(e)
#     return en_tweets

# en_train_pos_tweets = extract_engTweets(train_pos_tweets)
# en_train_neg_tweets = extract_engTweets(train_neg_tweets)
# en_test_pos_tweets = extract_engTweets(test_pos_tweets)
# en_test_neg_tweets = extract_engTweets(test_neg_tweets)
# print(len(train_pos_tweets))
# print(len(en_train_pos_tweets))
# print(len(train_neg_tweets))
# print(len(en_train_neg_tweets))
# print(len(test_pos_tweets))
# print(len(en_test_pos_tweets))
# print(len(test_neg_tweets))
# print(len(en_test_neg_tweets))


In [68]:
#pre processing
import re
import wordsegment
import autocorrect
from wordsegment import load, segment
load()
from autocorrect import Speller
spell=Speller()
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")
tknzr = TweetTokenizer()

punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
testTweet1='@PERKSOFNIALLJH RTed! Goood Luck :)'
testTweet2='My legs hurt so bad :))))))))))))))))))'
testTweet3='#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris trouble for being top engaged members in my community this week :)'
testTweet4='@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!'

def pre_process (tweets):
  result_tweets = []
  for tweet in tweets:  
    # print(f'raw tweet: {tweet}')
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove links that start with HTTP/HTTPS in the tweet
    tweet = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', "", tweet, flags=re.MULTILINE) # to remove other url links
    tweet = re.sub(r"RT", ' ', tweet, flags=re.MULTILINE) #remove 'RT'
    tweet = re.sub(r"#(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '#'
    tweet = re.sub(r"@(\w+)", ' ', tweet, flags=re.MULTILINE) #remove '@'
    tweet = ''.join([i for i in tweet if not i in punctuations]) #remove ponctuations
    tweet = ' '.join(segment(tweet)) #Remove repeated letters
    tweet = ' '.join([spell(w) for w in tweet.split()]) #Avoiding misspellings and slang words
    tweet = re.sub(r"\d", "", tweet) #remove digits
    tweet = tweet.lower()
    tokens = tknzr.tokenize(tweet) #tokenize
    tokens = [word for word in tokens if not word in stopwords.words()]
    tweet = (" ").join(stemmer.stem(token) for token in tokens)
    # print(f'preprocessed tweet: {tweet}')
    result_tweets.append(tweet)
  
  return result_tweets

# proc_en_train_pos_tweets = pre_process([testTweet1,testTweet2,testTweet3,testTweet4])
# proc_en_train_pos_tweets = pre_process([en_train_pos_tweets[0]])
# proc_en_train_neg_tweets = pre_process([en_train_neg_tweets[0]])
proc_train_pos_tweets = pre_process(train_pos_tweets)
proc_train_neg_tweets = pre_process(train_neg_tweets)
proc_test_pos_tweets = pre_process(test_pos_tweets)
proc_test_neg_tweets = pre_process(test_neg_tweets)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
print(len(train_pos_tweets))
print(len(test_pos_tweets))
print(len(train_neg_tweets))
print(len(test_neg_tweets))

3500
1500
3500
1500


In [70]:
import numpy as np
proc_train_tweets = np.concatenate((proc_train_pos_tweets, proc_train_neg_tweets))
proc_test_tweets = np.concatenate((proc_test_pos_tweets, proc_test_neg_tweets))
proc_pos_tweets = np.concatenate((proc_train_pos_tweets, proc_test_pos_tweets))
proc_neg_tweets = np.concatenate((proc_train_neg_tweets, proc_test_neg_tweets))
# proc_all_tweets = np.concatenate((proc_train_pos_tweets, proc_train_neg_tweets, 
#                                   proc_test_pos_tweets, proc_test_neg_tweets))
proc_all_tweets = np.concatenate((proc_train_pos_tweets, proc_test_pos_tweets, 
                                  proc_train_neg_tweets, proc_test_neg_tweets))

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=5, max_df=0.75, ngram_range=(1,3), max_features=10000)
# vectorizer = TfidfVectorizer(min_df=1, max_df=0.75, ngram_range=(1,3), max_features=10000)
features_train_pos = vectorizer.fit_transform(proc_train_pos_tweets).toarray()
features_train_neg = vectorizer.fit_transform(proc_train_neg_tweets).toarray()
features_test_pos = vectorizer.fit_transform(proc_test_pos_tweets).toarray()
features_test_neg = vectorizer.fit_transform(proc_test_neg_tweets).toarray()
################################################################################
features_train = vectorizer.fit_transform(proc_train_tweets).toarray()
features_test = vectorizer.fit_transform(proc_test_tweets).toarray()
features_pos = vectorizer.fit_transform(proc_pos_tweets).toarray()
features_neg = vectorizer.fit_transform(proc_neg_tweets).toarray()

features_all = vectorizer.fit_transform(proc_all_tweets).toarray()
# print(f'features_train_pos: {features_train_pos}')
# print(f'type(features_train_pos): {type(features_train_pos)}')

In [72]:
print(f'features_train.shape: {features_train.shape}')
print(f'features_test.shape: {features_test.shape}')
print(f'features_train.shape: {features_pos.shape}')
print(f'features_test.shape: {features_neg.shape}')
print(f'features_train_pos.shape: {features_train_pos.shape}')
print(f'features_train_neg.shape: {features_train_neg.shape}')
print(f'features_test_pos.shape: {features_test_pos.shape}')
print(f'features_test_neg.shape: {features_test_neg.shape}')

features_train.shape: (7000, 1430)
features_test.shape: (3000, 690)
features_train.shape: (5000, 1104)
features_test.shape: (5000, 1027)
features_train_pos.shape: (3500, 806)
features_train_neg.shape: (3500, 752)
features_test_pos.shape: (1500, 393)
features_test_neg.shape: (1500, 342)


In [73]:
# import numpy as np
# all_tweets = np.concatenate((features_train_pos,features_train_neg,features_test_pos,features_test_neg))
# train_features = np.concatenate((features_train_pos,features_train_neg))
# test_features = np.concatenate((features_test_pos,features_test_neg))

In [74]:
## Scoring and labeling the tweets
# First calculate the overall polarity score
# Assume only positive and negative
# Classify via the number of neg/pos words and emoticons

# 'tweet is expected to be an integer iterable' 
def polarity_score(tweet):
  score = 0
  for feature in tweet:
    score += feature
  return score

# 'pol_score' is expected to be a scalar
def determine_sentiment(pol_score):
  # Highly positive: +2 
  if pol_score >= 2 and pol_score <= 4:
    return 2 
  # Moderate positive: +1 
  if pol_score > 0 and pol_score < 2:
    return 1
  # Moderate negative: -1 
  if pol_score > -2 and pol_score < 0:
    return -1 
  # Highly negative: -2 
  if pol_score >= -4 and pol_score <= -2:
    return -2
  if pol_score == 0: 
    return 0
  else:#Tirar depois
    # return 2
    return 0

In [75]:
## Testing the balancing and scoring
# Classify the tweets
def score_tweets(tweets):
  tweet, length = tweets.shape
  tweets_class = []
  for i in range(tweet):
    score = polarity_score(tweets[i,:])
    sentiment = determine_sentiment(score)
    tweets_class.append(sentiment)
  return tweets_class

# all_tweets_classes = np.array(score_tweets(all_tweets))
# train_pos_classes = np.array(score_tweets(features_train_pos))
# train_neg_classes = np.negative(np.array(score_tweets(features_train_neg)))
# test_pos_classes = np.array(score_tweets(features_test_pos))
# test_neg_classes = np.negative(np.array(score_tweets(features_test_neg)))

pos_classes = np.array(score_tweets(features_pos))
# neg_classes = np.array(score_tweets(features_neg))
neg_classes = np.negative(np.array(score_tweets(features_neg)))

# train_classes = np.concatenate((train_pos_classes, train_neg_classes))
# test_classes = np.concatenate((test_pos_classes, test_neg_classes))
# train_classes = np.array(score_tweets(features_train))
# test_classes = np.array(score_tweets(features_test))
# all_tweets_features = np.concatenate((features_pos, features_neg))
all_tweets_classes = np.concatenate((pos_classes, neg_classes)) 
print(all_tweets_classes.size)

10000


In [76]:
from sklearn.model_selection import train_test_split
features_train, features_test, train_classes, test_classes = train_test_split(
    features_all, all_tweets_classes, test_size=0.3)

In [77]:
# #Table 4 - Polarity of tweets
# high_positive = 0
# moderate_positive = 0
# neutral = 0
# moderate_negative = 0
# high_negative = 0

# for t in all_tweets_classes:
#   if t == 2:
#     high_positive += 1
#   if t == 1:
#     moderate_positive += 1
#   if t == 0:
#     neutral += 1
#   if t == -1:
#     moderate_negative += 1
#   if t == -2:
#     high_negative += 1

# hp_ratio = (high_positive/all_tweets_classes.size)*100
# mp_ratio = (moderate_positive/all_tweets_classes.size)*100
# neutral_ratio = (neutral/all_tweets_classes.size)*100
# mn_ratio = (moderate_negative/all_tweets_classes.size)*100
# hn_ratio = (high_negative/all_tweets_classes.size)*100

# print(f'high_positive: {high_positive} ({hp_ratio}%)')
# print(f'moderate_positive: {moderate_positive} ({mp_ratio}%)')
# print(f'neutral: {neutral} ({neutral_ratio}%)')
# print(f'moderate_negative: {moderate_negative} ({mn_ratio}%)')
# print(f'high_negative: {high_negative} ({hn_ratio}%)')

In [78]:
#Table 4 - Polarity of positive tweets
high_positive = 0
moderate_positive = 0
part_pos_neutral = 0

pos_classes = np.array(score_tweets(features_pos))

for t in pos_classes:
  if t == 2:
    high_positive += 1
  if t == 1:
    moderate_positive += 1
  if t == 0:
    part_pos_neutral += 1

print(f'high_positive: {high_positive}')
print(f'moderate_positive: {moderate_positive}')
print(f'part_pos_neutral: {part_pos_neutral}')

high_positive: 1695
moderate_positive: 2876
part_pos_neutral: 429


In [79]:
#Table 4 - Polarity of negative tweets
moderate_negative = 0
high_negative = 0
part_neg_neutral = 0

neg_classes = np.array(score_tweets(features_neg))

for t in neg_classes:
  if t == 2:
    high_negative += 1
  if t == 1:
    moderate_negative += 1
  if t == 0:
    part_neg_neutral += 1

print(f'high_negative: {high_negative}')
print(f'moderate_negative: {moderate_negative}')
print(f'part_neg_neutral: {part_neg_neutral}')

high_negative: 1636
moderate_negative: 2948
part_neg_neutral: 416


In [80]:
high_positive + high_negative + moderate_positive + moderate_negative + part_pos_neutral + part_neg_neutral

10000

In [81]:
# Gambiarra table 4
neutral = part_pos_neutral + part_neg_neutral

hp_ratio = (high_positive/all_tweets_classes.size)*100
mp_ratio = (moderate_positive/all_tweets_classes.size)*100
neutral_ratio = (neutral/all_tweets_classes.size)*100
mn_ratio = (moderate_negative/all_tweets_classes.size)*100
hn_ratio = (high_negative/all_tweets_classes.size)*100

print(f'high_positive: {high_positive} ({hp_ratio}%)')
print(f'moderate_positive: {moderate_positive} ({mp_ratio}%)')
print(f'neutral: {neutral} ({neutral_ratio}%)')
print(f'moderate_negative: {moderate_negative} ({mn_ratio}%)')
print(f'high_negative: {high_negative} ({hn_ratio}%)')

high_positive: 1695 (16.950000000000003%)
moderate_positive: 2876 (28.76%)
neutral: 845 (8.450000000000001%)
moderate_negative: 2948 (29.48%)
high_negative: 1636 (16.36%)


In [82]:
# Decision tree implementation
from sklearn import tree

decision_tree = tree.DecisionTreeClassifier()
# X = features_train, y = train_classes

# features_train = vectorizer.fit_transform(proc_train_tweets).toarray()
# train_classes = np.concatenate((train_pos_classes, train_neg_classes))
# train_pos_classes = np.array(score_tweets(features_train_pos))
# train_neg_classes = np.negative(np.array(score_tweets(features_train_neg)))

decision_tree.fit(features_train, train_classes)
y_predict = decision_tree.predict(features_test)

# decision_tree.fit(train_features, train_classes)
# y_predict = decision_tree.predict(test_features)

In [83]:
print(features_train.shape)
print(train_classes.shape)
print(features_test.shape)

(7000, 1958)
(7000,)
(3000, 1958)


In [84]:
# # Random forest implementation
# from sklearn.ensemble import RandomForestClassifier

# random_forest = RandomForestClassifier(n_jobs=-1)
# random_forest.fit(features_train, train_classes)
# y_predict = random_forest.predict(features_test)

In [85]:
# # SVR implementation
# from sklearn.svm import SVR

# svr = SVR(C=1.0, epsilon=0.2)
# svr.fit(features_train, train_classes)
# y_predict = svr.predict(features_test)

In [86]:
# # Multinomial Logistic Regression (Softmax)
# from sklearn.linear_model import LogisticRegression

# softmax = LogisticRegression(multi_class='multinomial')
# softmax.fit(features_train, train_classes)
# y_predict = softmax.predict(features_test)

In [87]:
print(test_classes.shape)
print(y_predict.shape)
print(features_test.shape)

(3000,)
(3000,)
(3000, 1958)


In [88]:
print(y_predict[:10])

[-1  2 -1  1 -1 -1 -1  1 -1  0]


In [89]:
from sklearn.metrics import classification_report
y_true = test_classes
y_pred = y_predict 
target_names = ['High Negative', 'Moderate Negative', 'Neutral', 'Moderate Positive', 'High Positive']
# target_names = ['Neutral', 'Moderate Positive', 'High Positive']
print(classification_report(y_true, y_pred, target_names=target_names))

                   precision    recall  f1-score   support

    High Negative       0.51      0.54      0.53       475
Moderate Negative       0.62      0.60      0.61       887
          Neutral       0.67      0.97      0.79       264
Moderate Positive       0.62      0.60      0.61       848
    High Positive       0.61      0.51      0.55       526

         accuracy                           0.61      3000
        macro avg       0.61      0.64      0.62      3000
     weighted avg       0.61      0.61      0.60      3000



In [90]:
# from sklearn.metrics import classification_report
# y_predict = random_forest.predict(features_train)
# y_true = train_classes
# y_pred = y_predict 
# # target_names = ['High Negative', 'Moderate Negative', 'Neutral', 'Moderate Positive', 'High Positive']
# target_names = ['Neutral', 'Moderate Positive', 'High Positive']
# print(classification_report(y_true, y_pred, target_names=target_names))

In [56]:
# 10-fold cross-validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# clf = decision_tree ou random_forest ou svr ou softmax
scores = cross_val_score(decision_tree, features_train, train_classes, scoring='accuracy', cv=cv, n_jobs=-1)
# Report performance
print(f'scores.shape{scores.shape}')
print(f'scores{scores}')
print(f'mean score: {np.mean(scores)}')

scores.shape(10,)
scores[0.59714286 0.6        0.58857143 0.61571429 0.58571429 0.61285714
 0.59142857 0.59142857 0.58428571 0.61571429]
mean score: 0.5982857142857143
